In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
import matplotlib.font_manager

# print([k for k in rcParams.keys() if "figure" in k])
plt.rcParams.update(
    {
        "figure.figsize": (10, 5),
        "figure.max_open_warning": False,
        "text.usetex": True,
        "font.family": "sans-serif",
        "font.sans-serif": ["Helvetica"],
    }
)

In [ ]:
def matrix_slices(
    df_matrix,
    frequencies,
    distances,
    saveas="",
    ymin=None,
    ymax=None,
    start_freq=1000,
    start_dist=30,
):
    from ipywidgets import (
        interact,
        interactive,
        fixed,
        interact_manual,
        FloatSlider,
        IntSlider,
    )
    import ipywidgets as widgets
    from plotting_tools import save_fig

    def update_figure(slice_f, slice_d):
        fig, axs = plt.subplots(1, 2)
        fig.set_size_inches(10, 5)
        # axs[0].semilogy(distances, df_matrix[slice_f, :])
        axs[0].plot(distances, df_matrix[slice_f, :])
        axs[0].set_title(f"frequency = {frequencies[slice_f]:.0f} Hz")
        axs[0].set_xlabel("distance [cm]")
        axs[0].set_ylabel("amplitude [-]")
        axs[0].set_ylim(ymin, ymax)

        # axs[1].semilogy(frequencies, df_matrix[:, slice_d])
        axs[1].plot(frequencies, df_matrix[:, slice_d])
        axs[1].set_title(f"distance = {distances[slice_d]:.0f} cm")
        axs[1].set_xlabel("frequency [Hz]")
        axs[1].set_ylabel("amplitude [-]")
        axs[1].set_ylim(ymin, ymax)
        if saveas != "":
            save_fig(fig, saveas)
        plt.show()

    start_freq_index = np.argmin(np.abs(start_freq - frequencies))
    start_dist_index = np.argmin(np.abs(start_dist - distances))
    interactive_plot = interact(
        update_figure,
        slice_f=IntSlider(
            start_freq_index, min=0, max=len(frequencies) - 1, continuous_update=False
        ),
        slice_d=IntSlider(
            start_dist_index, min=0, max=len(distances) - 1, continuous_update=False
        ),
    )

In [ ]:
from simulation import generate_room

angles = [0, 20, 90]
fig, axs = plt.subplots(1, len(angles), squeeze=False)
fig.set_size_inches(5 * len(angles), 5)
for ax, angle in zip(axs.flatten(), angles):
    room = generate_room(distance_cm=10, azimuth_deg=angle, ax=ax)
    ax.set_title(f"yaw{angle}")
    ax.legend(loc="lower left")

# Distance-frequency matrix

In [ ]:
from simulation import get_dist_slice_pyroom, get_dist_slice_theory

azimuth_deg = 20
distances_cm = np.arange(10, 50)
frequency = 1000

Hs = get_dist_slice_pyroom(frequency, distances_cm=distances_cm, azimuth_deg=azimuth_deg)

# make sure Hs lie in reasonable range
Hs /= np.mean(Hs)

Hs_theo = get_dist_slice_theory(frequency, distances_cm=distances_cm, azimuth_deg=azimuth_deg)
fig, ax = plt.subplots()
for i in range(Hs.shape[1]):
    ax.plot(distances_cm, Hs[:, i], label=f"mic{i}", ls="-", color=f"C{i}")
    ax.plot(distances_cm, Hs_theo[:, i], ls=":", color=f"C{i}")
ax.set_yscale("log")
ax.legend()

In [ ]:
from calibration import fit_distance_slice

for chosen_mics in [[0], range(4)]:
    exp_data = Hs[:, chosen_mics]
    coeffs, d_slice, cost = fit_distance_slice(
        exp_data, distances_cm, azimuth_deg, frequency, chosen_mics,
        method='minimize', optimize_absorption=True, fit_one_gain=False
    )
    print(coeffs)
    fig, ax = plt.subplots()
    for i, mic in enumerate(chosen_mics):
        ax.plot(distances_cm, Hs[:, i], label=f"mic{mic}", ls="-", color=f"C{mic}")
        ax.plot(distances_cm, d_slice[:, i], ls=":", color=f"C{mic}")
    ax.legend()
    ax.set_title(f"optimization with mics: {chosen_mics}")

In [ ]:
from simulation import get_freq_slice_pyroom, WIDEBAND_FILE, create_wideband_signal
from crazyflie_description_py.parameters import N_BUFFER, FS

mic_idx = 1
distances = np.arange(10, 60)
frequencies = np.fft.rfftfreq(N_BUFFER, 1 / FS)

fname = "results/df_matrix_pyroom.pkl"
try:
    raise
    series_all = pd.read_pickle(fname)
    print("read", fname)
    np.testing.assert_allclose(series_all.distances, distances)
    np.testing.assert_allclose(series_all.frequencies, frequencies)
    df_matrix = series_all.df_matrix
except Exception as e:
    import progressbar
    df_matrix = np.zeros((len(frequencies), len(distances)))
    
    try:
        raise
        signal = np.load(WIDEBAND_FILE)
    except:
        print("creating wideband signal...")
        signal = create_wideband_signal(frequencies)
        np.save(WIDEBAND_FILE, signal)
        print(f"saved as {WIDEBAND_FILE}")

    with progressbar.ProgressBar(max_value=len(distances) - 1) as bar:
        for j, distance_cm in enumerate(distances):
            slice_mics = get_freq_slice_pyroom(frequencies, distance_cm, signal)
            df_matrix[:, j] = slice_mics[mic_idx]
            bar.update(j)
    series = pd.Series(
        {"df_matrix": df_matrix, "distances": distances, "frequencies": frequencies}
    )
    pd.to_pickle(series, fname)
    print("saved as", fname)

In [ ]:
min_freq = 100
max_freq = 5000
min_dist = 1
max_dist = 50
freq_start = int(min_freq / max(frequencies) * len(frequencies))
freq_end = int(max_freq / max(frequencies) * len(frequencies))
dist_start = int(min_dist / max(distances) * len(distances))
dist_end = int(max_dist / max(distances) * len(distances))

dist = distances[dist_start:dist_end]
freq = frequencies[freq_start:freq_end]

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
ax.pcolorfast(dist, freq, df_matrix[freq_start:freq_end, dist_start:dist_end])
plt.xlabel("distance [cm]")
plt.ylabel("frequency [Hz]")
matrix_slices(
    df_matrix[freq_start:freq_end, dist_start:dist_end], freq, dist, #ymin=8, ymax=20,
)

In [ ]:
from simulation import get_freq_slice_theory

df_matrix_theo = np.empty((len(frequencies), len(distances)))
for j, distance_cm in enumerate(distances):
    slice_mics = get_freq_slice_theory(frequencies, distance_cm)
    df_matrix_theo[:, j] = slice_mics[:, mic_idx]

In [ ]:
from plotting_tools import save_fig, add_colorbar

start_freq = (2000,)
start_dist = (30,)
fig, ax = plt.subplots()
fig.set_size_inches(5, 5)
im = ax.pcolorfast(dist, freq, df_matrix_theo[freq_start:freq_end, dist_start:dist_end])
add_colorbar(fig, ax, im, title="amplitude [-]")
ax.set_title("distance-frequency matrix for $\\theta=0$")
ax.axhline(start_freq, color="C1", label="distance slice")
ax.axvline(start_dist, color="C8", label="frequency slice")
ax.legend(framealpha=1.0)
ax.set_xlabel("distance [cm]")
ax.set_ylabel("frequency [Hz]")
save_fig(fig, "plots/distance-frequency-matrix.pdf")

matrix_slices(
    df_matrix_theo[freq_start:freq_end, dist_start:dist_end],
    freq,
    dist,
    start_freq=start_freq,
    start_dist=start_dist,
    saveas="plots/distance-frequency-slices.pdf",
)

### vertical frequency slice

In [ ]:
from inference import (
    get_probability_cost,
    get_probability_fft,
    get_probability_bayes,
)
from constants import SPEED_OF_SOUND
from geometry import get_deltas_from_global
from geometry import get_orthogonal_distance_from_global

# distance_slices = [5, 10, 20, 30, 40, 50]


min_frequency = 1000
max_frequency = 5000
n_freqs = 32
freq_cont = np.linspace(min_frequency, max_frequency, 1000)
freq = np.linspace(min_frequency, max_frequency, n_freqs)

distances_grid = np.arange(10, 100)
n_max = 100  # for zero-padding

# highest frequency that we can resolve
df = freq[1] - freq[0]
delta_max = SPEED_OF_SOUND / (2 * df) * 100
print("max attainable delta:", delta_max)
d = get_orthogonal_distance_from_global(
    deltas_cm=delta_max, azimuth_deg=0, mic_idx=mic_idx
)
print("max attainable distance:", d)

distance_slices = [30, d, 80]
#sigma_y = 1 # works with 1e-1, 1e-2: really good! with 1, it breaks down.
for sigma_y in [0] + list(np.logspace(-3, 0, 4)):
    size = (5, 7)
    fig_all, axs = plt.subplots(len(distance_slices), sharey=True, sharex=True)
    axs[0].set_title("frequency slices at different distances ($\\theta$=0)")
    fig_all.set_size_inches(*size)
    for i, d in enumerate(distance_slices):
        idx = np.argmin(np.abs(distances - d))
        slice_cont = get_freq_slice_theory(distance_cm=d, frequencies=freq_cont)[:, mic_idx]
        slice_f = get_freq_slice_theory(distance_cm=d, frequencies=freq)[:, mic_idx]

        slice_f += np.random.normal(scale=sigma_y, size=len(slice_f))

        d_fft, p_fft, diff = get_probability_fft(slice_f, freq, n_max=n_max)
        p_cost = get_probability_cost(slice_f, freq, distances_grid)
        d_bayes_sigma, p_bayes_sigma, diff = get_probability_bayes(
            slice_f, freq, n_max=n_max, sigma=sigma_y
        )
        d_bayes, p_bayes, diff = get_probability_bayes(
            slice_f, freq, n_max=n_max
        )

        axs[i].set_ylabel("amplitude [-]")
        axs[i].plot(freq_cont, slice_cont, label=f"{d:.0f}cm", color="C0")
        axs[i].scatter(freq, slice_f, color="C0")
        axs[i].legend(loc="upper right")

        fig = plt.figure()
        fig.set_size_inches(*size)
        plt.plot(d_fft, p_fft, marker="o", label="FFT")
        plt.plot(d_bayes, p_bayes, marker="x", label="Bayes")
        plt.plot(d_bayes_sigma, p_bayes_sigma, marker=".", label="Bayes, known sigma")
        plt.plot(distances_grid, p_cost, marker="o", label="optimization")
        plt.axvline(d, ls=":", color="black", label="real distance")
        plt.title(f"probabilities at distance {d:.0f}cm ($\\theta$=0)")
        plt.ylabel("probability [-]")
        plt.xlim(min(distances_grid), max(distances_grid))
        plt.legend(loc="lower right")
        # plt.ylim(0, 1)
        plt.yscale("log")
        plt.xlabel("orthogonal distance [cm]")
        plt.ylim(1e-6, 1)
        save_fig(fig, f"plots/simulation/study_fslice_{d:.0f}_{str(sigma_y).replace('.','-')}.pdf")
    axs[i].set_xlabel("frequency [Hz]")
    print("max distance", d_fft[-1])
    save_fig(fig_all, f"plots/simulation/study_fslices_{str(sigma_y).replace('.','-')}.pdf")

### combination of multiple mics

In [ ]:
from simulation import simulate_distance_estimator
from estimators import get_estimate

azimuth_deg = 0
distance_cm = 20

fig, ax = plt.subplots()

fig, ax_all = plt.subplots()
fig.set_size_inches(10, 5)

distance_estimator = simulate_distance_estimator(
    chosen_mics=range(4), azimuth_deg=azimuth_deg, distance_cm=distance_cm, ax=ax
)

ds, probs = distance_estimator.get_distance_distribution()
d = get_estimate(ds, probs)

ax_all.scatter(ds * 100, probs, label='all')
ax_all.axvline(d * 100, color='k')
assert np.abs(d * 100 - distance_cm) <= 1, (distance_cm, d * 100)

for mic in range(4):
    distance_estimator = simulate_distance_estimator(
        chosen_mics=[mic], azimuth_deg=azimuth_deg, distance_cm=distance_cm, ax=ax
    )
    ds, probs = distance_estimator.get_distance_distribution()
    d = get_estimate(ds, probs)
    
    ax_all.scatter(ds * 100, probs, label=f'mic{mic}', color=f'C{mic}')
    ax_all.axvline(d * 100, color=f'C{mic}')
    
ax_all.set_xlabel("wall distance [cm]")
ax_all.set_ylabel("probability [-]")
ax_all.legend()

distance_estimator.context.plot(distance=distance_cm * 1e-2, azimuth_deg=azimuth_deg)

In [ ]:
import itertools

for n_mics in range(1, 5):
    for chosen_mics in itertools.combinations(range(4), n_mics):
        de = simulate_distance_estimator(
            chosen_mics=chosen_mics, azimuth_deg=azimuth_deg, distance_cm=distance_cm
        )
        ds, probs = de.get_distance_distribution()
        d = get_estimate(ds, probs)

        fig, ax = plt.subplots()
        fig.set_size_inches(10, 5)
        ax.scatter(ds * 100, probs)
        ax.set_xlabel("wall distance [cm]")
        ax.set_ylabel("probability [-]")
        ax.axvline(d * 100)
        ax.set_title(chosen_mics)
        print(d*100 - distance_cm)
        #assert np.abs(d * 100 - distance_cm) < 1e-3

In [ ]:
import progressbar
results = pd.DataFrame(columns=['distance_cm', 'azimuth_deg', 'mics', 'error', 'estimate'])
distances_cm = [30] #np.arange(10, 50, step=10)
azimuths_deg = [-40] #np.arange(-180, 180, step=30)
n_mics_list = range(2, 5)

combinations = []
for n in n_mics_list:
    combinations += list(itertools.combinations(range(4), n))
n_combinations = len(combinations)
print(combinations)

max_value = len(distances_cm) * len(azimuths_deg) * n_combinations
i = 0
with progressbar.ProgressBar(max_value=max_value) as p:
    for distance_cm, azimuth_deg in itertools.product(distances_cm, azimuths_deg):
        for chosen_mics in combinations:
            de = simulate_distance_estimator(
                chosen_mics=chosen_mics, azimuth_deg=azimuth_deg, distance_cm=distance_cm
            )
            d = de.get_distance_estimate() * 100
            results.loc[len(results), :] = dict(
                distance_cm=distance_cm,
                azimuth_deg=azimuth_deg,
                mics=chosen_mics,
                error=abs(d-distance_cm),
                estimate=d
            )
            p.update(i); i += 1
            
import seaborn as sns
plt.figure()
i = 0
for labels, df in results.groupby(['mics', 'distance_cm']):
    plt.scatter(df.azimuth_deg, df.error, label=f"{labels}", color=f'C{i}')
    i += 1
plt.legend()

### angle estimation

In [ ]:
azimuth_deg = 30
distance_cm = 20
distance_gt = distance_cm * 1e-2

distance_estimator = simulate_distance_estimator(
    chosen_mics=range(4), azimuth_deg=azimuth_deg, distance_cm=distance_cm, ax=ax
)

fig, ax_all = plt.subplots()
fig.set_size_inches(10, 5)

distance_estimator = simulate_distance_estimator(
    chosen_mics=range(4), azimuth_deg=azimuth_deg, distance_cm=distance_cm, ax=ax
)
thetas, probs = distance_estimator.get_angle_distribution(
    distance_estimate_m=distance_gt
)
theta = get_estimate(thetas, probs)
ax_all.scatter(thetas, probs, label=f'all, estimate={theta}')
ax_all.axvline(theta, color=f'C{mic}')

fig, ax = plt.subplots()
for mic in range(4):
    distance_estimator = simulate_distance_estimator(
        chosen_mics=[mic], azimuth_deg=azimuth_deg, distance_cm=distance_cm, ax=ax
    )
    thetas, probs = distance_estimator.get_angle_distribution(
        distance_estimate_m=distance_gt
    )
    theta = get_estimate(thetas, probs)
    ax_all.scatter(thetas, probs, label=f'mic{mic}, estimate={theta}')
    ax_all.axvline(theta, color=f'C{mic}')
    
ax_all.set_xlabel("wall angle [deg]")
ax_all.set_ylabel("probability [-]")
ax_all.set_title(f"d {distance_cm}cm, yaw {azimuth_deg}deg")
ax_all.legend()
distance_estimator.context.plot(distance=distance_cm * 1e-2, azimuth_deg=azimuth_deg)

### non-vertical frequency slice

In [ ]:
from simulation import get_df_theory_simple
nominal_distance = 10
nominal_yaw = 0
relative_distances = np.random.normal(0, 1, len(freq))
absolute_yaws = np.random.normal(0, 1, len(freq))

deltas, d0 = get_deltas_from_global(
    azimuth_deg=nominal_yaw, distances_cm=distances, mic_idx=mic_idx
)
nominal_delta, d0 = get_deltas_from_global(nominal_yaw, nominal_distance, mic_idx)
deltas_noisy, d0 = get_deltas_from_global(
    nominal_yaw + absolute_yaws, nominal_distance + relative_distances, mic_idx
)

fig, ax = plt.subplots()
ax.pcolorfast(deltas, frequencies, df_matrix_theo[:-1, :-1])
ax.axvline(nominal_delta, color="C1")
ax.scatter(deltas_noisy, freq, color="C1")
ax.set_ylim(min(freq), max(freq))

df_vert = get_df_theory_simple(nominal_delta, freq, d0=d0)
slice_f_vert = df_vert.flatten()

df = get_df_theory_simple(deltas_noisy, freq, d0=d0)
slice_f = np.diag(df)

plt.figure()
plt.plot(freq, slice_f_vert, label="vertical")
plt.plot(freq, slice_f, label="noisy")
plt.legend()

probs_rel = get_probability_cost(
    slice_f,
    freq,
    distances_grid,
    relative_ds=relative_distances,
    absolute_yaws=absolute_yaws,
)
probs = get_probability_cost(slice_f, freq, distances_grid)
plt.figure()
plt.plot(distances_grid, probs, label="ignore mvmt")
plt.plot(distances_grid, probs_rel, label="account for mvmt")
plt.yscale("log")
plt.legend()

### distance slice

In [ ]:
from inference import get_approach_angle_fft
from simulation import factor_distance_to_delta
azimuth_deg = 20
n_methods = 2

gamma_deg = 30

frequency = 3000
relative_distances_cm = np.arange(20, step=1.0)
start_distance_cm = 20

start_distances_grid = [start_distance_cm] #np.arange(40, 60)
gammas_grid = np.arange(91)
n_max = 1000

distances_cm = start_distance_cm - relative_distances_cm * np.sin(
    gamma_deg / 180 * np.pi
)
for mic_idx in range(4):
    deltas_m, d0 = get_deltas_from_global(azimuth_deg, distances_cm, mic_idx)

    slice_d = get_df_theory_simple(deltas_m, [frequency], flat=True, d0=d0)

    gammas, probs, periods = get_approach_angle_fft(
        slice_d,
        frequency,
        relative_distances_cm,
        n_max=n_max,
    )

    period_simple = 2 * frequency / SPEED_OF_SOUND

    fig = plt.figure()
    plt.plot(periods/np.sin(gamma_deg/180*np.pi), 
             probs, 
             label='estimated', 
             color='C0')
    plt.axvline(period_simple, label='simple', color='C1')
    fig.set_size_inches(5, 5)
    plt.title(f'mic{mic_idx}')
    plt.legend()

In [ ]:
import itertools
from inference import get_approach_angle_fft, get_approach_angle_cost
from geometry import get_orthogonal_distance_from_global

azimuth_deg = 0
mic_idx = 1
n_methods = 2

gammas_deg = [90, 60, 30]
freqs = np.linspace(2000, 4000, 3)
relative_distances_cm = np.arange(20, step=1.0)
start_distance_cm = 50

start_distances_grid = [50] #np.arange(40, 60)
gammas_grid = np.arange(91)

sigma_relative_cm = 0 #1e-5 #1e-5
n_max = 1000

distances_max = np.linspace(start_distance_cm-max(relative_distances_cm), start_distance_cm, 100)
deltas_max, d0 = get_deltas_from_global(azimuth_deg, distances_max, mic_idx)

sigma_y = 1e-2 #0 
for sigma_y in [0] + list(np.logspace(-3, 0, 4)):
    size = (5, 7)
    fig_slices, ax_slices = plt.subplots(len(gammas_deg), sharex=True, sharey=True)
    fig_slices.set_size_inches(*size)

    for i_gamma, gamma_deg in enumerate(gammas_deg):
        distances_cm = start_distance_cm - relative_distances_cm * np.sin(
            gamma_deg / 180 * np.pi
        )
        deltas_m, d0 = get_deltas_from_global(azimuth_deg, distances_cm, mic_idx)

        d_m = np.abs(np.mean(np.diff(distances_cm))) * 1e-2
        D_m = (np.max(distances_cm) - np.min(distances_cm)) * 1e-2
        print(f"distances: {d_m:.3e} m, range: {D_m} m")
        print(
            f"usable freqs: {2*SPEED_OF_SOUND/D_m:.0f} - {SPEED_OF_SOUND/d_m:.0f} Hz"
        )

        print("ground truth:", gamma_deg)


        for i_freq, frequency in enumerate(freqs):
            label = f"$f$={frequency:.0f}Hz, $\\gamma$={gamma_deg}deg"
            slice_d = get_df_theory_simple(deltas_m, [frequency], flat=True, d0=d0)
            slice_d += np.random.normal(scale=sigma_y, size=len(slice_d))

            # plotting
            slice_cont = get_df_theory_simple(deltas_max, [frequency], flat=True, d0=d0)
            ax_slices[i_gamma].plot(deltas_max, slice_cont, color=f"C{i_freq}")
            ax_slices[i_gamma].scatter(deltas_m, slice_d, label=label, color=f"C{i_freq}")

            gammas_bayes, probs_bayes = get_approach_angle_fft(
                slice_d,
                frequency,
                relative_distances_cm,
                n_max=n_max,
                bayes=True,
                sigma=None,
            )
            gamma_bayes = gammas_bayes[np.argmax(probs_bayes)]
            gammas_bayes_sigma, probs_bayes_sigma = get_approach_angle_fft(
                slice_d,
                frequency,
                relative_distances_cm,
                n_max=n_max,
                bayes=True,
                sigma=sigma_y,
            )
            gamma_bayes_sigma = gammas_bayes_sigma[np.argmax(probs_bayes_sigma)]

            gammas_fft, probs_fft = get_approach_angle_fft(
                slice_d,
                frequency,
                relative_distances_cm,
                n_max=n_max,
                bayes=False,
                sigma=sigma_y,
            )
            gamma_fft = gammas_fft[np.argmax(probs_fft)]

            probs_cost = get_approach_angle_cost(
                slice_d,
                frequency,
                relative_distances_cm,
                start_distances_grid,
                gammas_grid,
                mic_idx=mic_idx,
                # ax=ax,
            )  # is of shape n_start_distances x n_gammas_grid

            #fig, ax = plt.subplots()
            #ax.pcolorfast(start_distances_grid, gammas_grid, probs_cost.T)
            #ax.set_xlabel("start distance [cm]")
            #ax.set_ylabel("gamma [deg]")
            #ax.set_title("cost")

            fig = plt.figure()
            fig.set_size_inches(*size)
            i = 0
            for method, gammas, probs in zip(
                ["FFT", "optimization", "Bayes", "Bayes, known sigma"],
                [gammas_fft, gammas_grid, gammas_bayes, gammas_bayes_sigma],
                [probs_fft, probs_cost, probs_bayes, probs_bayes_sigma],
            ):
                gamma_idx = np.argmax(probs)
                gamma_max = gammas[gamma_idx]
                plt.plot(gammas, probs, label=method, color=f"C{i}", marker="o")
                #plt.axvline(gamma_max, color=f"C{i}")
                i += 1
            plt.axvline(gamma_deg, color="black", ls=":", label="real angle")
            plt.yscale("log")
            plt.title("probabilities for " + label)
            plt.ylim(1e-4, 1)
            plt.legend(loc='upper right')
            plt.xlabel("approach angle $\\gamma$[deg]")
            plt.ylabel("probability [-]")
            save_fig(fig, f"plots/simulation/study_dslice_{gamma_deg:.0f}_{frequency:.0f}_{str(sigma_y).replace('.','-')}.pdf")
        ax_slices[i_gamma].legend(loc='upper left')
        ax_slices[i_gamma].set_ylabel("amplitude [-]")
    ax_slices[-1].set_xlabel("path difference $\\Delta$ [m]")
    save_fig(fig_slices, f"plots/simulation/study_dslices_{str(sigma_y).replace('.','-')}.pdf")

# Geometry study

In [ ]:
from geometry import Context

d_arr = np.arange(5, 50, step=5) # cm
yaw_arr = np.arange(-180, 180) # deg

mic_idx = 0
mics = np.c_[[0, 0,]].T # cm
source = np.r_[5, 0] # cm
context = Context(2, mics, source)

delta_matrix = np.empty((len(yaw_arr), len(d_arr)))
for j, d in enumerate(d_arr):
    deltas = context.get_delta(yaw_arr, d, mic_idx)
    delta_matrix[:, j] = deltas
    
#for i, y in enumerate(yaw_arr):
#    deltas = context.get_delta(y, d_arr, mic_idx)
#    delta_matrix[i, :] = deltas

fig = plt.figure()
fig.set_size_inches(5, 5)
for j, d in enumerate(d_arr):
    plt.plot(yaw_arr, delta_matrix[:, j], label=f"{d}cm")
plt.xlabel("wall angle $\\beta$ [deg]")
plt.ylabel("path difference $d_1-d_0$ [cm]")
l = plt.legend(
    bbox_to_anchor=[1.0, 1.0], loc="upper left", title="orthogonal \n distance $d$"
)
plt.setp(l.get_title(), multialignment="center")
plt.grid()

In [ ]:
import matplotlib as mpl
from plotting_tools import save_fig

delta_arr = np.arange(10, 100, step=10)
yaw_arr = np.arange(-180, 180)

d_matrix = np.empty((len(yaw_arr), len(delta_arr)))

for j, delta in enumerate(delta_arr):
    ds = context.get_total_distance(delta, yaw_arr, mic_idx=0)
    d_matrix[:, j] = ds
    
for i, azimuth_deg in enumerate(yaw_arr):
    ds = context.get_total_distance(delta_arr, azimuth_deg, mic_idx=0)
    d_matrix[i, :] = ds
    

fig = plt.figure()
fig.set_size_inches(5, 5)
for j, d in enumerate(delta_arr):
    plt.plot(yaw_arr, d_matrix[:, j], label=f"{d}cm")
plt.xlabel("wall angle $\\beta$ [deg]")
plt.ylabel("orthogonal distance $d$ [cm]")
plt.grid(which="both")
l = plt.legend(
    bbox_to_anchor=[1.0, 1.0], loc="upper left", title="path difference \n $d_1 - d_0$"
)
plt.setp(l.get_title(), multialignment="center")
save_fig(fig, "plots/theory_distances.png")

## Noise study

All below depends on the nominal distance from the wall. We could make median performance vs. distance error and distance from wall plots, and repeat the same plot at different amplitude noises.

Preliminary studies:

- Find correct parameters for attenuation and wall loss, given our experimental data. Will be different from wall to wall! We can get different wall estimate from pyroomacoustics. 

Noise sensitivity plots:

- Add noise on "where" we measure: We think we are at d, but actually measure at d+epsilon. Or we think we are at theta, but actually measure at theta+epsilon. Both lead to an error in path length, so we can add noise on the path length and then translate that noise to theta and/or distance errrors (on separate y labels)

- Add noise on amplitudes. Check how high this is in practice for different settings (with/without motors, hovering or fixed, different windowing techniques, with/without interpolation, etc.)

Separate plot: 

- Add noise on frequencies to make them non-uniform (but known)

In [ ]:
from simulation import get_deltas_from_global

mic_idx = 3
frequencies = np.linspace(1000, 5000, 100)
distances_cm = np.arange(50, step=5)
azimuth_deg = np.arange(-180, 180)

sigmas_distance = np.arange(20)
sigmas_yaw = np.arange(-180, 180)

fig, axs = plt.subplots(1, 2, sharey=True)
fig.set_size_inches(5, 5)
for d in distances_cm[:: len(distances_cm) // 10]:
    delta, d0 = get_deltas_from_global(0, d, mic_idx)
    delta_noisy, d0 = get_deltas_from_global(sigmas_yaw, d, mic_idx)
    sigmas_delta = 1e2 * abs(delta - delta_noisy)
    axs[0].plot(sigmas_yaw, sigmas_delta, label=f"{d}cm")
axs[0].set_xlabel("$\\sigma_\\theta [deg]$")
axs[0].set_ylabel("$\\sigma_\\Delta [cm]$")
axs[0].legend(loc="upper left", title="distance $d$")

for yaw in azimuth_deg[:: len(azimuth_deg) // 10]:
    delta, d0 = get_deltas_from_global(yaw, 0, mic_idx)
    delta_noisy, d0 = get_deltas_from_global(yaw, sigmas_distance, mic_idx)
    sigmas_delta = 1e2 * abs(delta - delta_noisy)
    axs[1].plot(sigmas_distance, sigmas_delta, label=f"{yaw}deg")
axs[1].set_xlabel("$\\sigma_d$ [cm]")
axs[1].legend(loc="upper left", title="angle $\\theta$")
axs[1].set_ylim(-1, 45)
save_fig(fig, "plots/simulation/noise_study.pdf")

## frequency slices

In [ ]:
from pandas_utils import filter_by_dict
from plotting_tools import save_fig

# TODO(FD) verify that below holds with a different gain too.

def get_cos_amplitudes(distances_cm):
    from simulation import WALL_ABSORPTION, GAIN
    from simulation import get_deltas_from_global
    from generate_simulation_results import YAW_DEG, MIC_IDX
    
    distances_cm = results_df.distance.unique()
    deltas_m, d0 = get_deltas_from_global(
        distances_cm=distances_cm, mic_idx=MIC_IDX, azimuth_deg=YAW_DEG
    )
    alpha0 = 1 / (4 * np.pi * d0)
    alpha1 = (1 - WALL_ABSORPTION) / (4 * np.pi * (deltas_m + d0))
    cos_amplitudes = GAIN * 2 * alpha0 * alpha1
    return cos_amplitudes


def plot_cos_amplitudes(axs, results_df):
    distances_cm = results_df.distance.unique()
    cos_amplitudes = get_cos_amplitudes(distances_cm)
    for i, f in enumerate([1.0, 1.5, 2.0]):
        for ax in axs.flatten():
            ax.plot(
                distances_cm,
                f * cos_amplitudes,
                color=f"C{i}",
                label=f"$\\sigma_y=${f}$a(d)$",
            )
    [ax.set_ylim(None, results_df.sigmay.max()) for ax in axs.flatten()]
    axs[0, -1].legend(loc="upper right", framealpha=1.0)


def plot_cuts(results_df, category_label="sigmadelta", xlabel="distance", num_cuts=3):
    methods = results_df.method.unique()
    fig, axs = plt.subplots(1, len(methods), squeeze=False, sharey=True)
    fig.set_size_inches(5 * len(methods), 5)
    for i, (method, sub_df) in enumerate(results_df.groupby("method")):
        categories = sub_df[category_label].unique()

        idx = np.linspace(1, len(categories) - 1, num_cuts).astype(int)
        for category in categories[idx]:
            df = sub_df.loc[sub_df[category_label] == category]
            if not len(df):
                print(f"did not find {category} in {sub_df[category_label].unique()}")
                continue
            medians = df.groupby(xlabel).error.median()
            axs[0, i].plot(medians.index, medians.values, marker="o")
        # axs[0, i].set_yscale('log')
        axs[0, i].set_title(f"{labels[method]}")
        axs[0, i].set_xlabel(labels[xlabel])  # ('distance $d$ [cm]')
        axs[0, i].legend(np.round(categories[idx], 1), title=labels[category_label])
    axs[0, 0].set_ylabel("median error [cm]")
    return fig, axs

In [ ]:
from plotting_tools import plot_error_distance, labels

name = "amplitude_noise"
results_df = pd.read_pickle(f"results/simulation/{name}.pkl")

fig, axs = plot_error_distance(
    results_df, column="sigmay", name=name + " $\\sigma_y$ [-]", aggfunc=np.nanmedian
)
plot_cos_amplitudes(axs, results_df)
save_fig(fig, f"plots/simulation/{name}_median.pdf")
for method, sub_df in results_df.groupby("method"):
    fig, axs = plot_error_distance(
        sub_df, column="sigmay", name=name + " $\\sigma_y$ [-]", aggfunc=np.nanmedian
    )
    plot_cos_amplitudes(axs, results_df)
    save_fig(fig, f"plots/simulation/{name}_{method}_median.pdf")

    fig, axs = plot_error_distance(
        sub_df, column="sigmay", name=name + " $\\sigma_y$ [-]", aggfunc=np.nanstd
    )
    plot_cos_amplitudes(axs, results_df)
    save_fig(fig, f"plots/simulation/{name}_{method}_std.pdf")

    fig, axs = plot_cuts(sub_df, category_label="distance", xlabel="sigmay", num_cuts=3)
    save_fig(fig, f"plots/simulation/{name}_{method}_slice.pdf")

In [ ]:
# name = 'delta_noise_high'
name = "delta_noise"
results_df = pd.read_pickle(f"results/simulation/{name}.pkl")

fig, axs = plot_error_distance(
    results_df, column="sigmadelta", name=name + " [cm]", aggfunc=np.nanmean
)
#save_fig(fig, f"plots/simulation/{name}_median.pdf")
fig, axs = plot_error_distance(
    results_df, column="sigmadelta", name=name + " [cm]", aggfunc=np.nanmedian
)
save_fig(fig, f"plots/simulation/{name}_median.pdf")
fig, axs = plot_error_distance(
    results_df, column="sigmadelta", name=name + " [cm]", aggfunc=np.nanstd
)
save_fig(fig, f"plots/simulation/{name}_std.pdf")

for method, sub_df in results_df.groupby("method"):

    fig, axs = plot_error_distance(
        sub_df, column="sigmadelta", name=name + " [cm]", aggfunc=np.nanmedian
    )
    axs[0, 0].set_title(f"median error of {labels[method]}")
    save_fig(fig, f"plots/simulation/{name}_{method}_median.pdf")

    fig, axs = plot_error_distance(
        sub_df, column="sigmadelta", name=name + " [cm]", aggfunc=np.nanstd
    )
    axs[0, 0].set_title(f"error std of {labels[method]}")
    save_fig(fig, f"plots/simulation/{name}_{method}_std.pdf")

In [ ]:
name = "delta_noise"
results_df = pd.read_pickle(f"results/simulation/{name}.pkl")
for method, sub_df in results_df.groupby("method"):
    sub_df = sub_df.loc[sub_df.distance >= 10]
    fig, axs = plot_cuts(
        sub_df, category_label="distance", xlabel="sigmadelta", num_cuts=10
    )
    save_fig(fig, f"plots/simulation/{name}_{method}_distance_cuts.pdf")

    fig, axs = plot_cuts(
        sub_df, category_label="sigmadelta", xlabel="distance", num_cuts=3
    )
    save_fig(fig, f"plots/simulation/{name}_{method}_delta_cuts.pdf")

In [ ]:
name = "frequency_noise"
results_df = pd.read_pickle(f"results/simulation/{name}.pkl")
vmin = results_df.error.min() / 2
vmax = results_df.error.max() / 2
for method, sub_df in results_df.groupby("method"):
    fig, axs = plot_error_distance(
        sub_df,
        column="sigmaf",
        name=name + " [Hz]",
        aggfunc=np.nanmedian,
        vmin=vmin,
        vmax=vmax,
    )
    save_fig(fig, f"plots/simulation/{name}_{method}_median.pdf")
    fig, axs = plot_error_distance(
        sub_df,
        column="sigmaf",
        name=name + " [Hz]",
        aggfunc=np.nanstd,
        vmin=vmin,
        vmax=vmax,
    )
    save_fig(fig, f"plots/simulation/{name}_{method}_std.pdf")

In [ ]:
name = "joint_noise"
results_df = pd.read_pickle(f"results/simulation/{name}.pkl")

chosen_sigmas = results_df.sigmay.unique()
chosen_sigmas = chosen_sigmas[:: len(chosen_sigmas) // 3]

for sigma_y in chosen_sigmas:
    print("plotting", sigma_y)

    sub_df = filter_by_dict(results_df, {"sigmay": sigma_y})
    for method, sub_df in sub_df.groupby("method"):
        # fig, axs = plot_error_distance(sub_df, column='sigmadelta', name='delta noise', aggfunc=np.nanstd)
        # fig.suptitle(f'amplitude noise: {sigma_y}')
        fig, axs = plot_error_distance(
            sub_df,
            column="sigmadelta",
            name="delta noise [cm]",
            aggfunc=np.nanmedian,
            vmax=30,
        )
        axs[0, 0].set_title(f"{labels[method]}, amplitude noise [-]: {sigma_y}")
        save_fig(
            fig,
            f'plots/simulation/{name}_{method}_{str(sigma_y).replace(".","-")}_median.pdf',
        )

## distance slices

In [ ]:
from plotting_tools import plot_error_gamma
name = "angle_noiseless"
results_df = pd.read_pickle(f"results/simulation/{name}.pkl")

for method, sub_df in results_df.groupby("method"):
    fig, ax = plot_error_gamma(
        sub_df, column="frequency", name="frequency [Hz]", aggfunc=np.nanstd
    )
    ax.set_title(labels[method])
    #save_fig(fig, f"plots/simulation/{name}_{method}_std.pdf")
    fig, ax = plot_error_gamma(
        sub_df, column="frequency", name="frequency [Hz]", aggfunc=np.nanmedian
    )
    save_fig(fig, f"plots/simulation/{name}_{method}_median.pdf")
    ax.set_title(labels[method])
sub_df

In [ ]:
def plot_all_frequencies(name, logy=False, column="", column_name=""):
    results_df = pd.read_pickle(f"results/simulation/{name}.pkl")
    for method, method_df in results_df.groupby(["method"]):
        n_freqs = len(method_df.frequency.unique())
        fig, axs = plt.subplots(1, n_freqs, sharey=True)
        fig.set_size_inches(3*n_freqs, 5)
        
        vmin = method_df.error.min()
        vmax = method_df.error.max()
        
        for i, (frequency, sub_df) in enumerate(method_df.groupby("frequency")):
            if i < n_freqs-1:
                colorbar = False
            else:
                colorbar=True
                
            plot_error_gamma(
                sub_df, column=column, name=column_name, aggfunc=np.nanmedian, 
                logy=logy,
                vmin=vmin, vmax=vmax,
                colorbar=colorbar,
                ax=axs[i], fig=fig
            )
            if i > 0:
                axs[i].set_ylabel('')
            axs[i].set_title(f'{frequency:.0f}Hz')
        fig.suptitle(labels[method])
        save_fig(fig, f"plots/simulation/{name}_{method}_median.pdf")

name = "angle_relative_noise"
plot_all_frequencies(name, column="sigmarelative", column_name="movement noise [cm]")

In [ ]:
name = "angle_amplitude_noise"
plot_all_frequencies(name, logy=True, column="sigmay", column_name="log of amplitude noise[-]")

In [ ]:
name = "angle_joint_noise"
results_df = pd.read_pickle(f"results/simulation/{name}.pkl")

f = 3000
for (method,frequency), freq_df in results_df.groupby(['method', 'frequency']):
    if frequency != f:
        continue
    n_noises = len(freq_df.sigmay.unique())
    vmin = freq_df.error.min()
    vmax = freq_df.error.max()
    fig, axs = plt.subplots(1, n_noises, sharey=True) 
    for i, (sigmay, sub_df) in enumerate(freq_df.groupby('sigmay')):
        if i < n_noises - 1:
            colorbar = False
        else:
            colorbar = True
        plot_error_gamma(
            sub_df, column="sigmarelative", name="movement noise [cm]", aggfunc=np.nanmedian,
            ax=axs[i], fig=fig, colorbar=colorbar, vmin=vmin, vmax=vmax
        )
        axs[i].set_ylabel('')
        axs[i].set_title(f'amplitude noise {sigmay}')
    axs[0].set_ylabel('movement noise [cm]')
    fig.suptitle(f"{labels[method]}, frequency {frequency:.0f}Hz")
    save_fig(fig, f"plots/simulation/{name}_{frequency:.0f}_{method}_median.pdf")
#plot_all_frequencies(name, logy=False, column="sigmarelative", column_name="amplitude noise[-]")

# Geometry test

In [ ]:
from geometry import *

distance = 3
azimuth_deg = 70 #+ mic_idx * 90

#context = Context.get_crazyflie_setup()
context = Context.get_standard_setup()

for mic_idx in range(4):
    #azimuth_deg = 45 + mic_idx * 90
    context.source = context.mics[mic_idx] * 2
    n = get_normal(distance, azimuth_deg)[:2]
    
    context.plot(normal=n)
    
    delta = context.get_delta_from_normal(n, mic_idx)
    
    distance_est = context.get_total_distance(delta, azimuth_deg, mic_idx)
    source_distance = context.get_source_distance(delta, azimuth_deg, mic_idx)
    azimuth_est = context.get_angles(delta, source_distance, mic_idx)
    print(azimuth_est)
    #assert delta_est == 2, delta_est
    #assert source_distance == 1, source_distance
    np.testing.assert_allclose(distance_est, distance)
    assert azimuth_deg in azimuth_est, (azimuth_est, azimuth_deg)